# Development of regression model to predict car prices

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
import pickle

In [3]:
df_cars = pd.read_csv('cars.csv',sep=';',index_col=0)

In [7]:
df_cars.describe()

,Miltal,Modellår,Tillverkningsår,price
count,212.000000,212.000000,212.000000,212.000000
mean,7341.837264,2013.726415,2013.481132,97126.066038
std,6561.961058,3.971541,3.946877,55617.456865
min,0.000000,2002.000000,2002.000000,3000.000000
25%,237.500000,2012.000000,2011.000000,54900.000000
50%,6700.000000,2014.000000,2014.000000,84900.000000
75%,11749.500000,2017.000000,2017.000000,148750.000000
max,24749.500000,2018.000000,2018.000000,313873.000000


In [74]:
linear_regression = LinearRegression()

In [75]:
y = df_cars['price']
feature_colums = ['Miltal','Modellår','Tillverkningsår']
X = df_cars[feature_colums]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [95]:
degree = 2
model = make_pipeline(PolynomialFeatures(degree), linear_regression)

r2s = []
for i in range(20):

    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    r2 = r2_score(y_test, y_predict)
    r2s.append(r2)
np.mean(r2s)

0.9248030418362674

In [96]:
model.fit(X, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [97]:
pickle.dump(model, open('model.pkl', 'wb'))

In [120]:
df_cars['predicted price'] = model.predict(df_cars[feature_colums])
df_cars['price rating'] = df_cars['price'] / df_cars['predicted price']

In [121]:
df_cars.sort_values(by = 'price rating')

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price comparison,price rating
80186761,Diesel,24749.5,2002.0,2002.0,Manuell,Peugeot Partner 190C 2.0 -02,7000.0,partner,23026.870916,0.303993,0.303993
79915956,Diesel,22249.5,2008.0,2008.0,Manuell,Peugeut partner boxline -08,14500.0,partner,39272.428022,0.369216,0.369216
80841956,Diesel,17610.0,2008.0,2008.0,Manuell,Citroën Berlingo 1.6 HDI 75 HK -08,14900.0,berlingo,27910.377531,0.533852,0.533852
80873335,Bensin,19749.5,2002.0,2002.0,Manuell,"Peugeot Partner 1,6 Dragkrok Takräcke -02",3000.0,partner,5350.472665,0.560698,0.560698
80457209,Diesel,15749.5,2008.0,2008.0,Manuell,Citroen Berlingo 75 HDI -08,15000.0,berlingo,26739.230253,0.560974,0.560974
80907805,Diesel,14749.5,2013.0,2012.0,Manuell,"Berlingo 1,6 e-hdi låg skatt 2041 kr/år 2 äga -13",32000.0,berlingo,54835.341306,0.583565,0.583565
80898432,Diesel,13500.0,2015.0,2014.0,Manuell,Citroën Berlingo Van 1.6 HDi Manuell 92hk -15,44900.0,berlingo,74592.136240,0.601940,0.601940
79829675,Diesel,3800.0,2015.0,2014.0,Manuell,Citroën Berlingo III 1.6 HDi Skåp/RESERVERAD -15,72500.0,berlingo,113396.164632,0.639351,0.639351
77154567,Diesel,5600.0,2015.0,2014.0,Manuell,"Citroën Berlingo 1,6 HDI Lågmil -15",69900.0,berlingo,102214.906957,0.683853,0.683853
80630631,Bensin,19749.5,2003.0,2003.0,Manuell,"Citroen Berlingo 1,6 Family -03",6000.0,berlingo,8675.675962,0.691589,0.691589


In [122]:
index = df_cars['header'] == 'Renault Kangoo 1,5 (6 växlad) -08'
df_cars.loc[index]

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price comparison,price rating
80790572,Diesel,15749.5,2008.0,2008.0,Manuell,"Renault Kangoo 1,5 (6 växlad) -08",45000.0,kangoo,26739.230253,1.682921,1.682921
